In [77]:
import nltk
import subprocess
import os
import json
from structure import *
import re
from tqdm import tqdm
import operator
import time
from gensim import corpora, models
import numpy as np
from sklearn import neighbors
from random import randint
import scipy

Далее функции для обработки параграфов, которые делают следующее:

* regexp_tokenize - с помощью регулярных выражений оставляем только слова, состоящие из символов латинского алфавита и кириллицы, а также цифры (в которых мб '.' или ','), все цифры заменены на цифру 1.
    
* remove_stop_words - удаляем стоп-слова
    
* normalize 
    
* tokenize

In [6]:
pattern = re.compile(r'(\d+[.|,]*)+')
tokenizer = RegexpTokenizer(r'[а-яА-ЯA-Za-z]+|1')
def regexp_tokenize(sentence):
    return tokenizer.tokenize(pattern.sub('1', sentence))

In [7]:
stop_words = get_stop_words('russian')
search_pattern = r'[^\w-]'
searcher = re.compile(search_pattern)
def remove_stop_words(tokens):
    return [token for token in tokens if not searcher.search(token) and token.lower() not in stop_words]

In [8]:
morph = pymorphy2.MorphAnalyzer()
def normalize(tokens):
    return [morph.parse(token)[0].normal_form for token in tokens]

In [9]:
def tokenize(sentence):
    """
    Get tokens of sentence. Uses `nltk.word_tokenize`.
    """
    return word_tokenize(sentence, language='russian')

In [10]:
points = []
is_correct = []
min_par_len = 100
dictionary = dict()
for i, filename in tqdm(enumerate(os.listdir('D:\\proger\\test\\txts_pickled\\'))):
    points.append([])
    is_correct.append([])
    doc = Document()
    doc.load('D:\\proger\\test\\txts_pickled\\' + filename)

    # представляем весь договор в виде списка из последовательных пунктов этого договора:

    #  title
    sentences = [s.sentence for s in doc.title.sentences]
    tok_sent = normalize(remove_stop_words(regexp_tokenize(' '.join(sentences))))
    points[i].append(tok_sent)
    is_correct[i].append(True)
    for word in tok_sent:
        dictionary[word] = dictionary.get(word, 0) + 1

    # preamble
    sentences = [s.sentence for s in doc.preamble.sentences]
    tok_sent = normalize(remove_stop_words(regexp_tokenize(' '.join(sentences))))
    points[i].append(tok_sent)
    is_correct[i].append(True)
    for word in tok_sent:
        dictionary[word] = dictionary.get(word, 0) + 1 
            
    
    # numered text
    chapters = doc.get_titles_of_chapters()
    for chapter in chapters:
        doc[chapter].get_sentences()
        for paragraph in doc[chapter].queue:
            par_sent = paragraph.sentences
            pnts = [pnt.sentence for pnt in par_sent]
            tok_sent = normalize(remove_stop_words(regexp_tokenize(' '.join(pnts))))
            points[i].append(tok_sent)
            is_correct[i].append(True)
            for word in tok_sent:
                dictionary[word] = dictionary.get(word, 0) + 1 

    # attachment
    length = 0
    attachment = []
    for sentence in doc.attachment.sentences:
        tok_sent = normalize(remove_stop_words(regexp_tokenize(' '.join(sentence.sentence))))
        if len(attachment) + len(tok_sent) < min_par_len:
            attachment.extend(tok_sent)
        else:
            points[i].append(attachment)
            is_correct[i].append(True)
            for word in attachment:
                dictionary[word] = dictionary.get(word, 0) + 1 
            attachment = tok_sent
            #newPoint = Point(i, sentence_number, sentence.sentence, True)
            #db_session.add(newPoint)
            #sentence_number = sentence_number + 1
    #db_session.commit()
with open('points_texttiling.pickle', 'wb') as f:
    pickle.dump(points, f)
with open('is_correct_texttiling.pickle', 'wb') as f:
    pickle.dump(is_correct, f)

34284it [7:34:17,  1.26it/s]


Удаляю слова, количество вхождений которых меньше 5:

In [132]:
t1 = time.time()
i = 0
j = 0
k = 0
print('len(points)', len(points))
for i, point in enumerate(tqdm(points)):
    while j < len(points[i]):
        while k < len(points[i][j]):
            if dictionary[points[i][j][k]] < 5:
                #print('points[', i, j, k, ']', points[i][j][k])
                del points[i][j][k]
            else:
                k += 1
        k = 0
        j += 1
    j = 0
t2 = time.time()
print('t =', t2 - t1)

len(points) 34161


100%|███████████████████████████████████████████████████████████████████████████| 34161/34161 [02:02<00:00, 277.76it/s]


t = 123.0307047367096


In [133]:
dictionary = dict()
for document in tqdm(points):
    for par in document:
        for word in par:
            if word in dictionary:
                dictionary[word] += 1 
            else:
                dictionary[word] = 1
sorted_dict = sorted(dictionary.items(), key=operator.itemgetter(1))
print(sorted_dict[0])

100%|███████████████████████████████████████████████████████████████████████████| 34161/34161 [01:43<00:00, 331.22it/s]


('кингисепп', 5)


Удаляю слова, которые являются одной буквой:

In [128]:
t1 = time.time()
i = 0
j = 0
k = 0
print('len(points)', len(points))
while i < len(points):
    while j < len(points[i]):
        while k < len(points[i][j]):
            if points[i][j][k] != '1' and len(points[i][j][k]) == 1 :
                del points[i][j][k]
            else:
                k += 1
        k = 0
        j += 1
    j = 0
    i += 1
t2 = time.time()
print(t2-t1)

len(points) 34181
123.07534313201904


Удаляем документы, в которых меньше 5 параграфов:

In [129]:
t1 = time.time()

i = 0
j = 0
k = 0
print('len(points)', len(points))
while i < len(points):
    if len(points[i]) < 5:
        del points[i]
        del is_correct[i]
        
    else:       
        i += 1
print('len(points)', len(points))
t2 = time.time()
print(t2-t1)

len(points) 34181
len(points) 34161
3.4437999725341797


Костыль, после обработки остались пустые параграфы на месте заголовка, поэтому удаляю их тут:

In [130]:
t1 = time.time()
i = 0
j = 0
while i < len(points):
    while j < len(points[i]):
        if points[i][j] == []:
            del points[i][j]
            del is_correct[i][j]
        else:    
            j += 1
    j = 0
    i += 1
t2 = time.time()
print(t2-t1)

5.461698532104492


In [134]:
with open('points_texttiling.pickle', 'wb') as f:
    pickle.dump(points, f)
with open('is_correct_texttiling.pickle', 'wb') as f:
    pickle.dump(is_correct, f)

In [63]:
texts = ['\n\n'.join([' '.join(par) for par in point]) for point in points]

In [64]:
tt = nltk.tokenize.TextTilingTokenizer()

Обрабатыаем договоры с помощью texttiling

In [65]:
t = []
for text in tqdm(texts[:10000]):
    t.append(tt.tokenize(text))


  0%|                                                                              | 6/10000 [00:05<2:19:13,  1.20it/s]
Exception in thread Thread-32:
Traceback (most recent call last):
  File "C:\tools\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\tools\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\tools\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [3:08:38<00:00,  1.13s/it]


In [45]:
for i in t[0]:
    print('________________________________________', i)

________________________________________ договор

оказание транспортный экспедиторский услуга санкт петербург 1 акционерный общество ржд логистика именовать дальнейший клиент лицо директор филиал акционерный общество ржд логистика санкт петербург сержантовый александр валентин действующий основание доверенность 1 1 1 апрель 1 зарегистрировать реестр нотариус 1 далее 1 сторона открытый акционерный общество порт усть луга транспортный компания оао пул транс именовать дальнейший экспедитор лицо генеральный евстафьев игорь юрий действующий основание устав сторона именовать вместе сторона отдельность сторона заключить настоящий договор далее договор нижеследующий

предмет договор

договор клиент заказывать обязываться оплатить экспедитор вознаграждение счёт клиент обязываться выполнить организовать выполнение услуга связанный перевозка доставка указанный клиент груз внутри российский федерация международный сообщение экспорт импорт транзит

конкретный перечень необходимый клиент услуга усло

клиент нести ответственность возникнуть порча груз вследствие удержание экспедитор произвести соответствие плата 1 договор экспедитор нести ответственность сохранность удерживать груз обеспечивать условие хранение препятствовать утрата груз часть любой внешний повреждение груз тара упаковка
________________________________________ 

клиент нести ответственность убыток причинить экспедитор три лицо связь нарушение обязанность предоставление достаточный достоверный информация указанный плата 1 договор

случай отказ клиент услуга договор обязать уведомить письменный вид экспедитор оплата последний полный объ ме фактически оказать услуга

порядок разрешение споровый

соблюдение претензионный порядок предусмотренный договор предъявление друг друг иск обязательно

случай прибытие груз пункт назначение ненадлежащий состояние повреждение недостача утрата уполномоченный представитель сторона составлять соответствующий акт установление расхождение количество качество выдача груз который фиксиров

In [ ]:
w2vmodel = models.Word2Vec.load('w2v.model')

In [66]:
sentences = [par for doc in points for par in doc]
t1 = time.time()

w2vmodel = models.Word2Vec(min_count=4)
w2vmodel.build_vocab(sentences)
w2vmodel.train(sentences, total_examples=w2vmodel.corpus_count, epochs=w2vmodel.iter)
w2vmodel.save('w2v1.model')

t2 = time.time()
print(t2 - t1)


C:\tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


684.9920234680176


Находим средний w2v вектор для каждого параграфа:

In [135]:
average_w2v = []
w2v_vectors = []
for i, text in enumerate(tqdm(t[:1000])):
    average_w2v.append([])
    w2v_vectors.append([])
    for j, par in enumerate(text):
        w2v_vectors[i].append([(w2vmodel[word], word) for word in par.split()]) 
        average_w2v[i].append(sum([w2v_vector[0] for w2v_vector in w2v_vectors[i][j]]) / len(par))

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]C:\tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:19<00:00, 51.73it/s]


In [ ]:
general_w2v = []
for word in tqdm(dictionary.keys()):
    print(word)
    general_w2v.append([w2vmodel[word], word])

In [139]:
for i, text in enumerate(w2v_vectors[:1]):
    for j, par in enumerate(text):
        close = [[0, ''] for _ in range(10)]
        # находим 10 ближайших слов к среднему вектору w2v 
        for k, word in enumerate(par):
            cos_vec = 1 - scipy.spatial.distance.cosine(w2v_vectors[i][j][k][0], average_w2v[i][j])
            if cos_vec > close[0][0]:
                for c in range(10):
                    if close[c][0] < cos_vec and c < 9:
                        if close[c+1][0] < cos_vec:
                            close[c] = close[c+1]
                        else: 
                            close[c] = [cos_vec, w2v_vectors[i][j][k][1]]
                    elif close[c][0] < cos_vec and c == 9:
                        close[c] = [cos_vec, w2v_vectors[i][j][k][1]]
        # находим взвешенное среднее для полученных 10 слов
        weighted_mean = sum([close[c][0] * w2vmodel[close[c][1]] for c in range(10)]) / sum((close[c][0] for c in range(10)))
        # находим 10 самых близких слов из всей коллекции к взвешенному среднему вектору
        close = [[0, ''] for _ in range(10)]
        for k, gen_w2w in enumerate(general_w2v):
            cos_vec = 1 - scipy.spatial.distance.cosine(gen_w2w[0], weighted_mean)
            if cos_vec > close[0][0]:
                for c in range(10):
                    if close[c][0] < cos_vec and c < 9:
                        if close[c+1][0] < cos_vec:
                            close[c] = close[c+1]
                        else: 
                            close[c] = [cos_vec, gen_w2w[1]]
                    elif close[c][0] < cos_vec and c == 9:
                        close[c] = [cos_vec, gen_w2w[1]]
        print('\n', close,'\n', t[i][j])

C:\tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



 [[0.64754974158631717, 'иван'], [0.65036065158186107, 'алексей'], [0.65127617545111305, 'владимир'], [0.6526231609786044, 'виктор'], [0.65277312615994765, 'михайло'], [0.65614950377236414, 'василий'], [0.67363812468234385, 'сергей'], [0.67408599961244375, 'анатолий'], [0.6834838333580151, 'александр'], [0.89104498629566409, 'акционерный']] 
 договор

оказание транспортный экспедиторский услуга санкт петербург 1 акционерный общество ржд логистика именовать дальнейший клиент лицо директор филиал акционерный общество ржд логистика санкт петербург сержантовый александр валентин действующий основание доверенность 1 1 1 апрель 1 зарегистрировать реестр нотариус 1 далее 1 сторона открытый акционерный общество порт усть луга транспортный компания оао пул транс именовать дальнейший экспедитор лицо генеральный евстафьев игорь юрий действующий основание устав сторона именовать вместе сторона отдельность сторона заключить настоящий договор далее договор нижеследующий

предмет договор

договор кл

 [[0.39130995278207426, 'переправка'], [0.39166923534233855, 'ответственнность'], [0.44110904176537646, 'ответст'], [0.44201976713196367, 'обязанность'], [0.44791047992838751, 'виновный'], [0.53374840745659091, 'венность'], [0.55401368536013906, 'отвественность'], [0.57085028134392102, 'ответственно'], [0.59429696377992813, 'ветственность'], [0.98104280867613691, 'ответственность']] 
 

ответственность сторона

ответственность экспедитор

экспедитор нести ответственность клиент неисполнение ненадлежащий исполнение обязанность договор соответствие действующий законодательство

экспедитор доказать нарушение обязательство вызвать ненадлежащий исполнение договор перевозка ответственность клиент определяться правило который экспедитор отвечать соответствующий перевозчик ответственность экспедитор ограничить ограничиваться ответственность перевозчик

 [[0.56948737657895443, 'багаж'], [0.57008038025917873, 'рейс'], [0.57795016294976653, 'танкер'], [0.57887418255891709, 'порту'], [0.5994496256

любой изменение дополнение договор действительный условие совершить письменный форма подписать надлежащий уполномоченный представитель сторона

 [[0.56322100400007247, 'посредством'], [0.56863099903274428, 'отправить'], [0.60493174706946828, 'телефакс'], [0.63368896201288227, 'курьер'], [0.64306580125486701, 'сообщение'], [0.66715332856393517, 'факсова'], [0.67694137481792993, 'оригинал'], [0.68780732862028937, 'почта'], [0.76126728468431692, 'факс'], [0.83544734445959679, 'факсимильный']] 
 

случай изменение наименование местонахождение почтовый банковский реквизит смена руководитель сторона договор обязать письменно сообщить сторона течение 1 пять день момент вступление действие указанный изменение предоставление копия подтверждать документ

договор документ касаться договор изготовить передать помощь средство электронный вычислительный техника факсимильный телексный связь иметь такой юридический сила подлинник исключение претензия сторона признавать факсимильный подпись руководител